In [1]:
import pandas as pd

In [7]:
df = pd.DataFrame({
    'name'    : ['Alice', 'Bob', 'Charlie', 'Charlie', 'Alice', 'Bob'],
    'item' : ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'],
    'number'    : [3, 2, 4, 3, 2, 1],
})

In [8]:
df

,name,item,number
0,Alice,aaa,3
1,Bob,bbb,2
2,Charlie,ccc,4
3,Charlie,ddd,3
4,Alice,eee,2
5,Bob,fff,1


通販の商品注文なのか、レストランに入って注文してるのか、どういうシチュエーションなんだろう。
まぁ良いや。あんまり考えずに作ったデータセットなので。
で、nameの値に応じて番号を振りたいとしよう。こんなふうに。

（注意：name_idの順序付けには条件が無いものとする。すなわち、出現順やアルファベット順でなくても良いとする。）


In [10]:
df['name_id'] = [0, 1, 2, 2, 0, 1]
df

,name,item,number,name_id
0,Alice,aaa,3,0
1,Bob,bbb,2,1
2,Charlie,ccc,4,2
3,Charlie,ddd,3,2
4,Alice,eee,2,0
5,Bob,fff,1,1


## 方法3

In [11]:
df = pd.DataFrame({
    'name'    : ['Alice', 'Bob', 'Charlie', 'Charlie', 'Alice', 'Bob'],
    'item' : ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'],
    'number'    : [3, 2, 4, 3, 2, 1],
})

In [22]:
df['name_id'] = df['name'].astype('category').cat.codes
df

,name,item,number,name_id
0,Alice,aaa,3,0
1,Bob,bbb,2,1
2,Charlie,ccc,4,2
3,Charlie,ddd,3,2
4,Alice,eee,2,0
5,Bob,fff,1,1


In [13]:
df['name'].astype('category').cat.codes

0    0
1    1
2    2
3    2
4    0
5    1
dtype: int8

確かにこれでできているけど、何でこれで実現できるのか分からなかった。ので、調べた。

In [15]:
df['name']

0      Alice
1        Bob
2    Charlie
3    Charlie
4      Alice
5        Bob
Name: name, dtype: object

In [14]:
df['name'].astype('category')

0      Alice
1        Bob
2    Charlie
3    Charlie
4      Alice
5        Bob
Name: name, dtype: category
Categories (3, object): [Alice, Bob, Charlie]

astype()を使ってdtypeを変換した。dtypeがcategoryに変わっている。
dtypeについては以下も参照。


https://linus-mk.hatenablog.com/entry/2019/08/18/150845

categoryとは何か? を調べようとしたけど、
公式ドキュメントとか見ても長い説明が書いてあったので諦めた。

カテゴリ型のseriesはcatを通じて色々な情報を取得できる、らしい（pandasライブラリ活用入門 p.159）

カテゴリにしてそのIDを取得するには、


In [19]:
df['name'].astype('category').cat.categories

Index(['Alice', 'Bob', 'Charlie'], dtype='object')

categoryにすると何が嬉しいのか？

3点メリットがある
・
・

pandasライブラリ活用入門 p.158（pdf的には200）  
https://pbpython.com/pandas_dtypes_cat.html   ：Practical Business Python （あ、名前聞いたことあるわ）    
公式は  https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html  の先頭に3項目列挙されてる


## 方法1

In [23]:
df = pd.DataFrame({
    'name'    : ['Alice', 'Bob', 'Charlie', 'Charlie', 'Alice', 'Bob'],
    'item' : ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'],
    'number'    : [3, 2, 4, 3, 2, 1],
})

In [27]:
df['name_id'] = df['name'].factorize()

ValueError: Length of values does not match length of index

あれ。何でだろう。

In [29]:
df['name'].factorize()

(array([0, 1, 2, 2, 0, 1]), Index(['Alice', 'Bob', 'Charlie'], dtype='object'))

In [36]:
type(df['name'].factorize())

tuple

In [31]:
df['name'].factorize()[0]

array([0, 1, 2, 2, 0, 1])

というわけで、factorize関数はSeriesではなくtupleを返してくる。

tupleの1番目は「Seriesのどの位置に、どの要素が入っているか」を示す数値のndarrayが返る。
2番めは「使われている要素一覧」を示すIndexが返る。
（公式ドキュメントに書いてあるとおりだが。）
なるほど、Seriesを上記の2つに「分解して」返してくるので、factorizeという関数名なのであろう。

というわけで、今回ほしいのはtupleの1番目なので、factorize()したあとに[0]を指定すればよい。


In [37]:
df['name_id'] = df['name'].factorize()[0]
df

,name,item,number,name_id
0,Alice,aaa,3,0
1,Bob,bbb,2,1
2,Charlie,ccc,4,2
3,Charlie,ddd,3,2
4,Alice,eee,2,0
5,Bob,fff,1,1


## 方法2

In [41]:
df = pd.DataFrame({
    'name'    : ['Alice', 'Bob', 'Charlie', 'Charlie', 'Alice', 'Bob'],
    'item' : ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'],
    'number'    : [3, 2, 4, 3, 2, 1],
})

In [43]:
df['name_id'] = df.groupby(['name']).ngroup()
df

,name,item,number,name_id
0,Alice,aaa,3,0
1,Bob,bbb,2,1
2,Charlie,ccc,4,2
3,Charlie,ddd,3,2
4,Alice,eee,2,0
5,Bob,fff,1,1


https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.ngroup.html
